In [1]:
%load_ext autoreload
%autoreload 2 


In [33]:
from datetime import datetime, timedelta
from control.modules.thrusters import ThrustersControl, ThrustersParameters
from input_output.base import Stamped, StampedDf
from input_output.definitions.simulation import Boundary, HeatSource, TemperatureBoundary
from input_output.modules.thrusters import ThrustersControlValues, ThrustersSensorValues, ThrustersSimulationInputs, ThrustersSimulationOutputs
from orchestration.simulator import Simulator, SimulatorModel
import polars as pl 

start_time = datetime.fromtimestamp(0)
duration = timedelta(minutes=59)

#seawater_temperature = StampedDf.stamp(pl.DataFrame({'time':pl.datetime_range(start_time, start_time + duration, interval ='1m', time_unit = "us", eager=True), 'value':30*[32.0] + 30*[5.0]}))
supply_temperature = StampedDf.stamp(pl.DataFrame({'time':pl.datetime_range(start_time, start_time + duration, interval ='1m', time_unit = "us", eager=True), 'value':30*[50.0] + 30*[80.0]}))

simulation_inputs = ThrustersSimulationInputs(
    thrusters_aft=HeatSource(heat_flow=Stamped.stamp(9000.0)),
    thrusters_fwd=HeatSource(heat_flow=Stamped.stamp(4300.0)),
    thrusters_seawater_supply=Boundary(
        temperature=Stamped.stamp(32.0), flow=Stamped.stamp(64.0)),
    thrusters_module_supply=TemperatureBoundary(temperature=supply_temperature))

control = ThrustersControl(ThrustersParameters(cooling_mix_setpoint=40, cooling_pump_dutypoint=0.9, recovery_pump_dutypoint=0.5, max_temp=70, recovery_mix_setpoint=60))
control.to_recovery() 

thrusters_model = SimulatorModel(
    fmu_path=str(
        "../src/simulation/models/thrusters/thruster_moduleV6.fmu"
    ),
    sensor_values_cls=ThrustersSensorValues,
    control_values_cls=ThrustersControlValues,
    simulation_outputs_cls=ThrustersSimulationOutputs,
    simulation_inputs=simulation_inputs,
    control = control,
    tick_duration = timedelta(seconds = 10),
    start_time=start_time)

simulation = Simulator(thrusters_model)

result = await simulation.run(6*60)


[WARNING] Warning: The following was detected at time: 3.37221189657358E-29
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_mixing_return: Flow splitRatio is far above 1. But it must be between 0 and 1. (is 1.00639)
  Failed condition: thrusterModule.flowJoin_mixing_return.splitRatio <= 1.001

[WARNING] Warning: The following was detected at time: 6.459465245663797E-26
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_thruster_out: Flow splitRatio is negative and far below 0. But it must be between 0 and 1. (is -3.30264e+15)
  Failed condition: thrusterModule.flowJoin_thruster_out.splitRatio >= -0.001

[WARNING] Warning: The following was detected at time: 1.439614660203414E-12
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_AFT: Flow splitRatio is far above 1. But it must be between 0 and 1. (is 1.001)
  Failed condition: thrusterModule.flowJoin_AFT.splitRatio <= 1.001

[WARNING] Warning: The following was detected at time: 6.185076671684502E-06
[WARNING]   Thrus

/Users/alje/Repos/zero-thrs-control/src/input_output/base.py:110: UserWarning: Time 1970-01-01 01:59:10 is before than the given range of data for field thrusters_module_supply.
  warn(
/Users/alje/Repos/zero-thrs-control/src/input_output/base.py:110: UserWarning: Time 1970-01-01 01:59:20 is before than the given range of data for field thrusters_module_supply.
  warn(
/Users/alje/Repos/zero-thrs-control/src/input_output/base.py:110: UserWarning: Time 1970-01-01 01:59:30 is before than the given range of data for field thrusters_module_supply.
  warn(
/Users/alje/Repos/zero-thrs-control/src/input_output/base.py:110: UserWarning: Time 1970-01-01 01:59:40 is before than the given range of data for field thrusters_module_supply.
  warn(
/Users/alje/Repos/zero-thrs-control/src/input_output/base.py:110: UserWarning: Time 1970-01-01 01:59:50 is before than the given range of data for field thrusters_module_supply.
  warn(


In [34]:
#simulate scenario: start in recovery, increase thrusters source (trigger warm-up mixing), increase seawater temp (trigger cooling mode) 
#collect control modes and plot
#collect simulation inputs and plot 

#look up mixture and its specific heat capacity

In [35]:
def compute_heat(temp1, temp2, flow):
    return (temp1-temp2)*flow*4184/60

In [1]:
import polars as pl 

df = result.group_by_dynamic("time", every="1m").agg([
    pl.col("*").mean()
])

# Add new columns for heat calculations
heat = pl.DataFrame({"thrusters_heat_recovered":
    compute_heat(
        df["thrusters_module_return__temperature__C"],
        df["thrusters_module_supply__temperature__C"],
        df["thrusters_module_supply__flow__l_min"]
    ),
    "thrusters_heat_dumped" :compute_heat(
        df["thrusters_seawater_return__temperature__C"],
        df["thrusters_seawater_supply__temperature__C"],
        df["thrusters_seawater_supply__flow__l_min"]
    ),
    "thrusters_aft_heat":compute_heat(
        df["thrusters_temperature_aft_return__temperature__C"],
        df["thrusters_temperature_supply__temperature__C"],
        df["thrusters_flow_aft__flow__l_min"]
    ),
    "thrusters_fwd_heat":compute_heat(
        df["thrusters_temperature_fwd_return__temperature__C"],
        df["thrusters_temperature_supply__temperature__C"],
        df["thrusters_flow_fwd__flow__l_min"]
    )
})

heat_df = heat.with_columns(df['time']).to_pandas().set_index('time')
temp_df = df.select([col for col in df.columns if "__temperature__" in col or col=="time"]).to_pandas().set_index('time')
flow_df = df.select([col for col in df.columns if "__flow__" in col or col=="time"]).to_pandas().set_index('time')
valve_df = df.select([col for col in df.columns if ("__position_rel__" in col and 'mix' in col) or col=="time"]).to_pandas().set_index('time')

NameError: name 'result' is not defined

In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt


frames = [heat_df, temp_df, flow_df, valve_df]
fig, axes = plt.subplots(len(frames), 1, figsize=(10, 8), sharex=True)

for i, df in enumerate(frames):
    sns.lineplot(df, linewidth = 2.5, ax = axes[i])
    axes[i].legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.tight_layout()


NameError: name 'heat_df' is not defined